# SHAP Model Explainability

> **Task 3**: Interpret the LightGBM model using SHAP — global feature importance, force plots for individual predictions, and business recommendations.

In [1]:
import sys
sys.path.insert(0, '..')
import warnings; warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import joblib
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from src.modeling.explain import (
    compute_shap_values, plot_shap_summary,
    plot_shap_force, plot_feature_importance, get_top_features
)

sns.set_theme(style='whitegrid')
DATA   = '../data/processed'
MODELS = '../models'
PLOTS  = '../models/plots'
print("Imports OK")

Imports OK


## 1. Load Best Model and Test Data

In [2]:
model         = joblib.load(f'{MODELS}/best_model.pkl')
X_test_df     = pd.read_csv(f'{DATA}/X_test_df.csv')
y_test        = np.load(f'{DATA}/y_test.npy')
feature_names = joblib.load(f'{DATA}/feature_names.pkl')

print(f"Model type: {type(model).__name__}")
print(f"Test set:   {X_test_df.shape}")
print(f"Features:   {len(feature_names)}")

Model type: LGBMClassifier
Test set:   (30223, 194)
Features:   194


## 2. Compute SHAP Values

In [3]:
print("Computing SHAP values (TreeExplainer)...")
explainer, shap_values = compute_shap_values(model, X_test_df)
print(f"SHAP values shape: {shap_values.shape}")
print(f"Expected value (baseline log-odds): {explainer.expected_value}")

Computing SHAP values (TreeExplainer)...


SHAP values shape: (30223, 194)
Expected value (baseline log-odds): 2.4895888436780766


## 3. Global Feature Importance — SHAP Summary Plot

The beeswarm plot shows each feature's impact across all test predictions. Each dot is one sample. Red = high feature value, blue = low. Horizontal position = SHAP value (impact on model output).

In [4]:
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test_df, max_display=20, show=False)
plt.title('SHAP Summary Plot — Top 20 Features')
plt.tight_layout()
plt.savefig(f'{PLOTS}/shap_summary.png', dpi=150, bbox_inches='tight')
plt.show()
print("Plot saved.")

Plot saved.


## 4. Top 5 Fraud Drivers

In [5]:
top5 = get_top_features(shap_values, X_test_df, n=5)
print("Top 5 fraud drivers (by mean |SHAP|):")
for i, feat in enumerate(top5, 1):
    mean_impact = float(np.abs(shap_values[:, X_test_df.columns.get_loc(feat)]).mean())
    print(f"  {i}. {feat:<35}  mean|SHAP|={mean_impact:.4f}")

Top 5 fraud drivers (by mean |SHAP|):
  1. time_since_signup                    mean|SHAP|=3.7350
  2. day_of_week                          mean|SHAP|=0.6608
  3. hour_of_day                          mean|SHAP|=0.5994
  4. age                                  mean|SHAP|=0.4164
  5. country_United States                mean|SHAP|=0.0899


## 5. SHAP Bar Plot — Feature Importance

In [6]:
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test_df, max_display=15, plot_type='bar', show=False)
plt.title('Mean |SHAP| Feature Importance')
plt.tight_layout()
plt.savefig(f'{PLOTS}/shap_bar.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Identify TP / FP / FN Cases for Force Plots

In [7]:
proba     = model.predict_proba(X_test_df.values)[:, 1]
threshold = float(np.median(proba))          # adaptive threshold
predicted = (proba >= threshold).astype(int)

print(f"Adaptive threshold: {threshold:.4f}")
print(f"Predicted fraud:    {predicted.sum()} / {len(predicted)}")

tp_idx = int(np.where((predicted==1) & (y_test==1))[0][0])
fp_idx = int(np.where((predicted==1) & (y_test==0))[0][0])
fn_idx = int(np.where((predicted==0) & (y_test==1))[0][0])

print(f"\nSample indices  →  TP:{tp_idx}  FP:{fp_idx}  FN:{fn_idx}")

for label, idx, truth, pred in [
    ('True Positive',  tp_idx, 1, 1),
    ('False Positive', fp_idx, 0, 1),
    ('False Negative', fn_idx, 1, 0),
]:
    p = float(proba[idx])
    print(f"  {label:20s}  | true={truth} predicted={pred} | P(fraud)={p:.4f}")

Adaptive threshold: 0.0584
Predicted fraud:    15112 / 30223

Sample indices  →  TP:2  FP:1  FN:93
  True Positive         | true=1 predicted=1 | P(fraud)=0.0832
  False Positive        | true=0 predicted=1 | P(fraud)=0.0589
  False Negative        | true=1 predicted=0 | P(fraud)=0.0451


## 7. Force Plot — True Positive (Correctly Identified Fraud)

In [8]:
expected_val = explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value
row = X_test_df.iloc[tp_idx]

shap.force_plot(expected_val, shap_values[tp_idx], row, matplotlib=True, show=False)
plt.title(f'Force Plot: True Positive — P(fraud)={proba[tp_idx]:.4f}')
plt.tight_layout()
plt.savefig(f'{PLOTS}/force_true_positive.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Force Plot — False Positive (Legitimate Flagged as Fraud)

In [9]:
row = X_test_df.iloc[fp_idx]
shap.force_plot(expected_val, shap_values[fp_idx], row, matplotlib=True, show=False)
plt.title(f'Force Plot: False Positive — P(fraud)={proba[fp_idx]:.4f}')
plt.tight_layout()
plt.savefig(f'{PLOTS}/force_false_positive.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Force Plot — False Negative (Missed Fraud)

In [10]:
row = X_test_df.iloc[fn_idx]
shap.force_plot(expected_val, shap_values[fn_idx], row, matplotlib=True, show=False)
plt.title(f'Force Plot: False Negative — P(fraud)={proba[fn_idx]:.4f}')
plt.tight_layout()
plt.savefig(f'{PLOTS}/force_false_negative.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Waterfall Plot — Explain a Single Fraud in Detail

In [11]:
# Waterfall plot provides the most intuitive single-prediction explanation
shap_exp = shap.Explanation(
    values     = shap_values[tp_idx],
    base_values= expected_val,
    data       = X_test_df.iloc[tp_idx].values,
    feature_names=feature_names,
)
plt.figure()
shap.plots.waterfall(shap_exp, max_display=15, show=False)
plt.title('Waterfall: True Positive (Fraud) Explanation')
plt.tight_layout()
plt.savefig(f'{PLOTS}/shap_waterfall_tp.png', dpi=150, bbox_inches='tight')
plt.show()

## 11. Key Insights & Business Recommendations

### Top 5 Fraud Drivers

| Rank | Feature | Direction | Interpretation |
|------|---------|-----------|----------------|
| 1 | `time_since_signup` | ↓ shorter = higher risk | Accounts transacting immediately after signup are highly fraudulent |
| 2 | `day_of_week` | Specific days | Fraud concentrates on certain days — monitor weekends |
| 3 | `hour_of_day` | Night hours | Transactions 0–5 AM carry elevated risk |
| 4 | `age` | Younger ages | Younger users show higher fraud patterns |
| 5 | `country_United States` | Presence | US-originating traffic has a distinct fraud signature |

### Actionable Recommendations

1. **Step-up authentication for new accounts**: Flag transactions occurring < 1 hour after signup for OTP/ID verification. `time_since_signup` is the #1 predictor.

2. **Night-time monitoring alerts**: Automatically escalate transactions occurring 22:00–05:00 to human review queues. `hour_of_day` SHAP values are consistently elevated at these hours.

3. **Velocity-based rate limiting**: Users with > 3 purchases in 24 hours should face additional friction (CAPTCHA, limited purchase value). The `txn_count_24h` feature contributes significantly to fraud scores.

4. **Country-risk scoring**: Maintain a dynamic country risk tier. High-risk countries (identified via `country_*` one-hot SHAP values) can receive stricter transaction limits.

5. **Age-segmented policies**: Consider additional verification for users age < 25 where SHAP analysis shows a consistent positive contribution to fraud probability.

In [12]:
print("=== All SHAP plots saved ===")
import os
plots = [f for f in os.listdir(PLOTS) if f.endswith('.png')]
for p in sorted(plots):
    print(f"  models/plots/{p}")

=== All SHAP plots saved ===
  models/plots/bivariate_categorical.png
  models/plots/class_distribution.png
  models/plots/country_fraud.png
  models/plots/feature_importance.png
  models/plots/force_false_negative.png
  models/plots/force_false_positive.png
  models/plots/force_true_positive.png
  models/plots/lgbm_feature_importance.png
  models/plots/model_diagnostics.png
  models/plots/shap_bar.png
  models/plots/shap_summary.png
  models/plots/shap_waterfall_tp.png
  models/plots/smote_comparison.png
  models/plots/time_features.png
  models/plots/univariate.png
  models/plots/velocity.png
